In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

from torch import nn
import torch
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Config:
    device = torch.device("cuda")
    MAX_SEQ = 100
    EMBED_DIMS = 512
    ENC_HEADS = DEC_HEADS = 8
    NUM_ENCODER = NUM_DECODER = 4
    BATCH_SIZE = 32
    TRAIN_FILE = "/opt/ml/input/data/train_data.csv"
    TEST_FILE = "/opt/ml/input/data/test_data.csv"
    TOTAL_EXE = 13523
    TOTAL_CAT = 10000



# 1.데이터 불러오기

기존 SAINT : train의 Column 종류

row_id, timestamp, user_id, content_id, content_type_id,

task_container_id, user_answer, answered_correctly, prior_question_elapsed_time, prior_question_had_explanation

우리 DKT

userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag

공통적으로 User id, timestamp 있고

콘텐츠적으로 나누면 우리 dkt 는 3개정도 분류될거 같고,

saint+이기 때문에 time 에 관해서 더 프로세싱하면 될거 같다.(elapsed, Lagged time 은 데이터상으로 구별하기 힘들다.)


전처리


'timestamp': 'int64',  -> Timestamp

'user_id': 'int32',  -> userID

'content_id': 'int16' -> assessmentItemID

'answered_correctly': 'int8',   -> answerCode



"content_type_id": "int8", -> 0으로 생성



"prior_question_elapsed_time": "float32", -> Timestamp 변형

"task_container_id": "int16" -> 누적 개수 (새로 생성 필요)



In [3]:
print("loading csv.....")
train_df = pd.read_csv(Config.TRAIN_FILE)
test_df = pd.read_csv(Config.TEST_FILE)

test_df = test_df[test_df['answerCode']!=-1]
train_df = pd.concat([train_df, test_df])

train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

print("shape of (train) dataframe :", train_df.shape)

train_df['grade'] = (train_df['assessmentItemID'].str[2]).astype('int')
train_df['testId'] = (train_df['assessmentItemID'].str[2] + train_df['assessmentItemID'].str[4:7])
train_df["Timestamp"] = pd.to_datetime(train_df["Timestamp"])

ItemID2idx = {v:k for k,v in enumerate(train_df['assessmentItemID'].unique())} # 9453개
testId2idx = {v:k for k,v in enumerate(train_df['testId'].unique())} # 1536개
Tag2idx = {v:k for k,v in enumerate(train_df['KnowledgeTag'].unique())} # 911개

train_df['assessmentItemID'] = train_df['assessmentItemID'].map(ItemID2idx)
train_df['testId'] = train_df['testId'].map(testId2idx)
train_df['KnowledgeTag'] = train_df['KnowledgeTag'].map(Tag2idx)

#필요 Column 생성 - 'prior_question_elapsed_time'
train_df['prior_question_elapsed_time'] = 0

for i in range(len(train_df)-1):
    if train_df.at[i,'userID'] == train_df.at[(i+1),'userID']:
        train_df.at[i,'prior_question_elapsed_time'] = (train_df.at[i+1,'Timestamp']-train_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'train elapsetime process  {i}/{len(train_df)} done ')

train_df = train_df.drop(['Timestamp'], axis=1)

loading csv.....
shape of (train) dataframe : (2525956, 6)


KeyboardInterrupt: 

In [9]:
#data augmentation 증강 수 aug에 입력하자
aug = 10


print(f'======origin length      : {len(train_df)}======')
train_origin = train_df.copy()
train_aug = train_df.copy()
n= 1
for i in range(aug):
    print(f'START {n}th AUGMENTATION')
    tem = train_origin.drop_duplicates(subset = ["userID"],keep = "last")
    train_origin = train_origin.drop(index=tem.index)
    train_origin['userID'] += 7442 #train_origin['userID'].nunique()+1
    train_aug = pd.concat([train_aug, train_origin], axis = 0)
    print(f'END   {n}th AUGMENTATION')
    n += 1
print(f'======after augmentation : {len(train_aug)}======')

======origin length      : 2525956======
START 1th AUGMENTATION
END   1th AUGMENTATION
START 2th AUGMENTATION
END   2th AUGMENTATION
START 3th AUGMENTATION
END   3th AUGMENTATION
START 4th AUGMENTATION
END   4th AUGMENTATION
START 5th AUGMENTATION
END   5th AUGMENTATION
START 6th AUGMENTATION
END   6th AUGMENTATION
START 7th AUGMENTATION
END   7th AUGMENTATION
START 8th AUGMENTATION
END   8th AUGMENTATION
START 9th AUGMENTATION
END   9th AUGMENTATION
START 10th AUGMENTATION
END   10th AUGMENTATION
======after augmentation : 27376207======


In [8]:
train_aug.to_csv(f'/opt/ml/input/data/train_saint_dkt_x{aug}.csv',index = False)

## test csv 만들기

In [27]:
print("loading csv.....")
test_df = pd.read_csv(Config.TEST_FILE)
print("shape of (train) dataframe :", train_df.shape)


test_df['grade'] = (test_df['assessmentItemID'].str[2]).astype('int')
test_df['testId'] = test_df['assessmentItemID'].str[2] + test_df['assessmentItemID'].str[4:7]
test_df["Timestamp"] = pd.to_datetime(test_df["Timestamp"])


ItemID2idx = {v:k for k,v in enumerate(test_df['assessmentItemID'].unique())} # 9453개
testId2idx = {v:k for k,v in enumerate(test_df['testId'].unique())} # 1536개
Tag2idx = {v:k for k,v in enumerate(test_df['KnowledgeTag'].unique())} # 911개

test_df['assessmentItemID'] = test_df['assessmentItemID'].map(ItemID2idx)
test_df['testId'] = test_df['testId'].map(testId2idx)
test_df['KnowledgeTag'] = test_df['KnowledgeTag'].map(Tag2idx)



#필요 Column 생성 - 'prior_question_elapsed_time'
test_df['prior_question_elapsed_time'] = 0


for i in range(len(test_df)-1):
    if test_df.at[i,'userID'] == test_df.at[(i+1),'userID']:
        test_df.at[i,'prior_question_elapsed_time'] = (test_df.at[i+1,'Timestamp']-test_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'test elpasetime process  {i}/{len(test_df)} done ')
        
test_df = test_df.drop(['Timestamp'], axis=1)


loading csv.....
shape of (train) dataframe : (74876098, 7)
test elpasetime process  0/260114 done 
test elpasetime process  100000/260114 done 
test elpasetime process  200000/260114 done 


In [28]:
test_df.to_csv(f'/opt/ml/input/data/test_saint_dkt.csv',index = False)


# train test 병합 안하기

In [8]:
print("loading csv.....")
train_df = pd.read_csv(Config.TRAIN_FILE)
test_df = pd.read_csv(Config.TEST_FILE)
print("shape of (train) dataframe :", train_df.shape)

# 기본적인 데이터 전처리 부분.
# column : userID assessmentItemID  testId answerCode Timestamp KnowledgeTag grade

train_df['grade'] = (train_df['assessmentItemID'].str[2]).astype('int')
train_df['testId'] = (train_df['assessmentItemID'].str[2] + train_df['assessmentItemID'].str[4:7])
train_df["Timestamp"] = pd.to_datetime(train_df["Timestamp"])

test_df['grade'] = (test_df['assessmentItemID'].str[2]).astype('int')
test_df['testId'] = test_df['assessmentItemID'].str[2] + test_df['assessmentItemID'].str[4:7]
test_df["Timestamp"] = pd.to_datetime(test_df["Timestamp"])

ItemID2idx = {v:k for k,v in enumerate(train_df['assessmentItemID'].unique())} # 9453개
testId2idx = {v:k for k,v in enumerate(train_df['testId'].unique())} # 1536개
Tag2idx = {v:k for k,v in enumerate(train_df['KnowledgeTag'].unique())} # 911개

# ItemID2idx = {v:k for k,v in enumerate(test_df['assessmentItemID'].unique())} # 9453개
# testId2idx = {v:k for k,v in enumerate(test_df['testId'].unique())} # 1536개
# Tag2idx = {v:k for k,v in enumerate(test_df['KnowledgeTag'].unique())} # 911개


train_df['assessmentItemID'] = train_df['assessmentItemID'].map(ItemID2idx)
train_df['testId'] = train_df['testId'].map(testId2idx)
train_df['KnowledgeTag'] = train_df['KnowledgeTag'].map(Tag2idx)

test_df['assessmentItemID'] = test_df['assessmentItemID'].map(ItemID2idx)
test_df['testId'] = test_df['testId'].map(testId2idx)
test_df['KnowledgeTag'] = test_df['KnowledgeTag'].map(Tag2idx)



# #필요 Column 생성 - 'content_type_id'
# train_df['content_type_id'] = 0
# test_df['content_type_id'] = 0

#필요 Column 생성 - 'prior_question_elapsed_time'
train_df['prior_question_elapsed_time'] = 0
test_df['prior_question_elapsed_time'] = 0

for i in range(len(train_df)-1):
    if train_df.at[i,'userID'] == train_df.at[(i+1),'userID']:
        train_df.at[i,'prior_question_elapsed_time'] = (train_df.at[i+1,'Timestamp']-train_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'train elapsetime process  {i}/{len(train_df)} done ')

for i in range(len(test_df)-1):
    if test_df.at[i,'userID'] == test_df.at[(i+1),'userID']:
        test_df.at[i,'prior_question_elapsed_time'] = (test_df.at[i+1,'Timestamp']-test_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'test elpasetime process  {i}/{len(test_df)} done ')
        
train_df = train_df.drop(['Timestamp'], axis=1)
test_df = test_df.drop(['Timestamp'], axis=1)

# # 필요 Column 생성 - 'task_container_id'
# train_df['task_container_id'] = 0
# test_df['task_container_id'] = 0

# for i in range(1,len(train_df)):
#     if train_df.at[i,'userID'] == train_df.at[i-1,'userID']:
#         train_df.at[i,'task_container_id'] = train_df.at[i-1,'task_container_id'] + 1
#     if i %100000 == 0:
#         print(f'train task process  {i}/{len(train_df)} done ')

# for i in range(1,len(test_df)):
#     if test_df.at[i,'userID'] == test_df.at[i-1,'userID']:
#         test_df.at[i,'task_container_id'] = test_df.at[i-1,'task_container_id'] + 1
#     if i %100000 == 0:
#         print(f'test task process  {i}/{len(test_df)} done ')

# # SAINT에 필요한 것만 남김
# train_df = train_df.drop(['testId','KnowledgeTag','grade'], axis=1)
# test_df = test_df.drop(['testId','KnowledgeTag','grade'], axis=1)

# # change to SAINT Input
# train_df  = train_df.rename(columns = {'userID': 'user_id', 'Timestamp':'timestamp', 'assessmentItemID' : 'content_id',
#                            'answerCode' : 'answered_correctly'  })
# test_df  = test_df.rename(columns = {'userID': 'user_id', 'Timestamp':'timestamp', 'assessmentItemID' : 'content_id',
#                            'answerCode' : 'answered_correctly'  })


loading csv.....
shape of (train) dataframe : (2266586, 6)
train elapsetime process  0/2266586 done 
train elapsetime process  100000/2266586 done 
train elapsetime process  200000/2266586 done 
train elapsetime process  300000/2266586 done 
train elapsetime process  400000/2266586 done 
train elapsetime process  500000/2266586 done 
train elapsetime process  600000/2266586 done 
train elapsetime process  700000/2266586 done 
train elapsetime process  800000/2266586 done 
train elapsetime process  900000/2266586 done 
train elapsetime process  1000000/2266586 done 
train elapsetime process  1100000/2266586 done 
train elapsetime process  1200000/2266586 done 
train elapsetime process  1300000/2266586 done 
train elapsetime process  1400000/2266586 done 
train elapsetime process  1500000/2266586 done 
train elapsetime process  1600000/2266586 done 
train elapsetime process  1700000/2266586 done 
train elapsetime process  1800000/2266586 done 
train elapsetime process  1900000/2266586 do

In [11]:
#data augmentation 증강 수 aug에 입력하자
aug = 30


print(f'======origin length      : {len(train_df)}======')
train_origin = train_df.copy()
train_aug = train_df.copy()
n= 1
for i in range(aug):
    print(f'START {n}th AUGMENTATION')
    tem = train_origin.drop_duplicates(subset = ["userID"],keep = "last")
    train_origin = train_origin.drop(index=tem.index)
    train_origin['userID'] += 7442 #train_origin['userID'].nunique()+1
    train_aug = pd.concat([train_aug, train_origin], axis = 0)
    print(f'END   {n}th AUGMENTATION')
    n += 1
print(f'======after augmentation : {len(train_aug)}======')

======origin length      : 2266586======
START 1th AUGMENTATION
END   1th AUGMENTATION
START 2th AUGMENTATION
END   2th AUGMENTATION
START 3th AUGMENTATION
END   3th AUGMENTATION
START 4th AUGMENTATION
END   4th AUGMENTATION
START 5th AUGMENTATION
END   5th AUGMENTATION
START 6th AUGMENTATION
END   6th AUGMENTATION
START 7th AUGMENTATION
END   7th AUGMENTATION
START 8th AUGMENTATION
END   8th AUGMENTATION
START 9th AUGMENTATION
END   9th AUGMENTATION
START 10th AUGMENTATION
END   10th AUGMENTATION
START 11th AUGMENTATION
END   11th AUGMENTATION
START 12th AUGMENTATION
END   12th AUGMENTATION
START 13th AUGMENTATION
END   13th AUGMENTATION
START 14th AUGMENTATION
END   14th AUGMENTATION
START 15th AUGMENTATION
END   15th AUGMENTATION
START 16th AUGMENTATION
END   16th AUGMENTATION
START 17th AUGMENTATION
END   17th AUGMENTATION
START 18th AUGMENTATION
END   18th AUGMENTATION
START 19th AUGMENTATION
END   19th AUGMENTATION
START 20th AUGMENTATION
END   20th AUGMENTATION
START 21th AUGMEN

In [10]:
train_aug.to_csv(f'/opt/ml/input/data/train_saint_dkt_notest_x{aug}.csv',index = False)